In [1]:
import pandas as pd
existing_data = pd.read_excel('output.xlsx')
recent_date = existing_data['Data anunt'].max()

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from openpyxl import Workbook
from bs4 import BeautifulSoup
import re

#setam calea către driverul Chrome și deschidem browserul
PATH = "C:\\Users\\Larisa\\Desktop\\licenta\\chromedriver2" 
driver = webdriver.Chrome(PATH)

# deschide site-ul Blitz.ro și caută anunțuri de vânzare
url= "https://www.blitz.ro/cluj-napoca/vanzari-apartamente"
#url = "https://www.blitz.ro/cluj-napoca?utm_term=imobiliare%20cluj&utm_campaign=14484281179&utm_content=&utm_source=google&utm_medium=cpc&gclid=Cj0KCQiAjbagBhD3ARIsANRrqEtb1A5VEDDrbYpvgAd42_6u8Fc3HznDCHRUrpb0QIjVnquUbfhr59IaAr_gEALw_wcB"
driver.get(url)


anunturi = []
table = {}

count = 0
cards = driver.find_elements(By.CSS_SELECTOR, '.card')
for i in range(len(driver.find_elements(By.CSS_SELECTOR, '.card__content'))):
    try:
         # condiția de ieșire din buclă
        if count == 90:
            break
        # accesam linkul pentru fiecare anunț și extragem informațiile dorite din codul sursă
        card = driver.find_elements(By.CSS_SELECTOR, '.card__content')[i]
        link = card.find_element(By.CSS_SELECTOR, '.card__content--head a').get_attribute('href')
        print(link)
        driver.get(link)
        
        offer_features = driver.find_element(By.CSS_SELECTOR, '.offer-features')
        soup = BeautifulSoup(offer_features.get_attribute('innerHTML'), 'html.parser')
        
        offer_info = driver.find_element(By.CSS_SELECTOR, '.offer-informations')
        soup2 = BeautifulSoup(offer_info.get_attribute('innerHTML'), 'html.parser')
        
        anunt ={}

        # Find all the <strong> tags and use their text as column names
        for strong in soup.find_all('strong'):
            column_name = strong.text.strip()
            anunt[column_name] = ''

      #  offer_info = soup2.find('div', class_='offer-informations')
        if offer_info is not None:
            for item in soup2.find_all('div', class_='offer-informations--item'):
                strong_tags = item.find_all('strong')
                if strong_tags is not None:
                    for index, strong_tag in enumerate(strong_tags):
                        if 'Cartier : ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                print(column_value)
                                anunt['Cartier'] = column_value
                        elif 'Pret/mp: ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                                if numeric_value:
                                    column_value = numeric_value.group(0).replace('.', '')  #eliminare "." estetice
                                    column_value = column_value.replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                                print(column_value)
                                anunt['Pret/mp'] = column_value 
                        elif 'Actualizat:' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                anunt['Data anunt'] = column_value  
                            
        for li in soup.find_all('li'):
            for strong in li.find_all('strong'):
               
                column_name = strong.text.strip()
                column_value = ''
                if strong.next_sibling is not None:
                    column_value = strong.next_sibling.strip()
                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                if numeric_value:
                    column_value = numeric_value.group(0).replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                    print(column_value)    
                #table[column_name].append(column_value)
                anunt[column_name] = column_value

        print(anunt) 
        #conditie pentru adaugarea anunturilor cu o data mai actuala decta cele deja existente
        if pd.to_datetime(anunt['Data anunt']) > pd.to_datetime(recent_date):
            anunturi.append(anunt)
        
        
        driver.back()  # revenim la pagina de căutare pentru a continua să parcurgeți anunțuri
    except NoSuchElementException:
        continue
    count += 1 

df2 = pd.DataFrame(anunturi)
new_data = pd.DataFrame(anunturi)
df_final = pd.concat([existing_data, new_data], ignore_index=True)
# Print the dataframe and export it to Excel
print(df_final)
df_final.to_excel('output.xlsx', index=False)

# inchidem browserul
driver.quit()    

C:\Users\Larisa\AppData\Local\Temp\ipykernel_2656\2547831997.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-buna-ziua-blitz118068av
Bună Ziua
2355
2
55
1
1
1
2008
{'Nr. camere:': '2', 'Suprafață utilă': '55', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '1', 'Boxă la subsol:': 'Da', 'Tip compartimentare:': 'Decomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Mobilat/utilat', 'Anul construcției:': '2008', 'Tip finisaj:': 'Finisat', 'Modalitate vânzare:': 'Credit bancar, Cash', 'Facilități:': '', 'Data anunt': '10.06.2023', 'Cartier': 'Bună Ziua', 'Pret/mp': '2355'}


C:\Users\Larisa\AppData\Local\Temp\ipykernel_2656\2547831997.py:96: UserWarning: Parsing '29.05.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  if pd.to_datetime(anunt['Data anunt']) > pd.to_datetime(recent_date):


https://www.blitz.ro/cluj-napoca/garsoniera-vanzare-floresti-blitz118352av
Floreşti
1349
43
1
1
1
2010
{'Nr. camere:': 'Garsonieră', 'Suprafață utilă': '43', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Vechi', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Mobilat/utilat', 'Anul construcției:': '2010', 'Tip finisaj:': 'Finisat', 'Materiale construcție:': 'Beton, Cărămidă', 'Modalitate vânzare:': 'Credit bancar, Cash', 'Facilități:': '', 'Data anunt': '10.06.2023', 'Cartier': 'Floreşti', 'Pret/mp': '1349'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-semicentral-blitz118347av
Semicentral
3385
2
57.61
1
1
19
1
2023
{'Nr. camere:': '2', 'Suprafață utilă': '57.61', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '19', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'În construcție', 'Tip imobil:': 'Bloc d

https://www.blitz.ro/cluj-napoca/apartament-vanzare-3-camere-baciu-blitz90461av
Baciu
1500
3
80
2
1
3
1
1990
{'Nr. camere:': '3', 'Suprafață utilă': '80', 'Nr. băi:': '2', 'Nr. balcoane:': '1', 'Etaj:': '3', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '1990', 'Tip finisaj:': 'Semifinisat', 'Facilități:': '', 'Data anunt': '09.06.2023', 'Cartier': 'Baciu', 'Pret/mp': '1500'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-floresti-blitz109833av
Floreşti
1416
2
55
1
1
2
1
2022
{'Nr. camere:': '2', 'Suprafață utilă': '55', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '2', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Modern/lux', 'Anul construcției:': '2022', 'Tip finisaj:': 'Modern/lux

Manastur
2000
2
57
1
1
6
1
2018
{'Nr. camere:': '2', 'Suprafață utilă': '57', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '6', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Modern/lux', 'Anul construcției:': '2018', 'Tip finisaj:': 'Finisat', 'Materiale construcție:': 'Cărămidă', 'Modalitate vânzare:': 'Credit bancar, Cash', 'Facilități:': '', 'Data anunt': '09.06.2023', 'Cartier': 'Manastur', 'Pret/mp': '2000'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-iris-blitz106685av
Iris
1452
2
42
1
1
4
2010
{'Nr. camere:': '2', 'Suprafață utilă': '42', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '4', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '2010', 'Tip finisaj:': 'Finisat', 'Modalitate vânzare:': 'Credit 